# Consumer-Product Matrix

A Consumer-product matrix is an $m \times n$ matrix $C$, where each row represents a consumer, and each column corresponds to a product. The element $c_{ij}$ in this matrix represents the probability that consumer $i$ will buy or like product $j$.

To put it simply, our goal is to gain insights into the factors influencing people's purchasing decisions. We want to use this understanding to predict their future choices, even when we lack complete information.

We are assuming that certain hidden characteristics, such as age, gender, income, etc., impact consumers' buying decisions, and the decision of each consumer is only a function of these hidden features.

With this hypothesis we can rewrite $C = AB$. The matrix $A$ reflects the extent to which hidden features influence each consumer's choices, and the matrix $B$ provides information about the probability of a consumer buying or liking a product based on a specific hidden feature.

In an ideal scenario, we'd have complete data in our large table. However, in reality, data gaps are common, and our goal is to predict missing information. This is where challenges like the Netflix challenge come into play, where we are given some ratings and tasked with predicting ratings for other movies. In online advertising, we aim to determine which ad is best for a user based on their past purchases.

In this lab, we usebest k rank for Movie Recommendations (for more information, please refer to the related videos posted on Moodle for this week).

Instructions:

**Step 1:** Data Gathering

**Step 2:** Data Preprocesing

**Step 3:** The best k rank predicts ratings!!!

**Step 4:** Writing a function to recommend movies for any user.

**Step 1: Data Gathering:**

1. Start by importing the necessary Python libraries, such as Numpy and Pandas.
2. Next, visit the provided URL: http://grouplens.org/datasets/movielens/. Under the "recommended for education and development" section, locate and download the file named `ml-latest-small.zip` (which has a size of 1 MB).
3. After downloading, import the CSV files contained within the zip file.


In [ ]:
import numpy as np
import pandas as pd

links = pd.read_csv('links.csv')
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv')

print(ratings)

        userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]


**Step 2: Data Preprocessing:**

1. I began by examining the first few rows of my data to familiarize myself with its structure.
2. Next, I transformed the data so that each row represents a user via the `.pivot()` function.
3. Note that 'NaN' values in the dataset represent missing or unrated movies by users. Common treatment to handle these 'NaN' values include replacing them with zero or the average rating for each row or column. So, decided to use fillna(0) so that it wouldn't recommend movies that haven't been reviewed before.
4. I converted this transformed table into a numerical matrix (C).
5. I also decided that feature normalization wasn't that necessary for this dataset.


In [ ]:
# you code

ratings_1 = ratings.pivot(index='userId', columns='movieId', values='rating')
ratings_2 = ratings_1.fillna(0)
C = ratings_2.to_numpy()
print(C)

[[4.  0.  4.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [2.5 2.  2.  ... 0.  0.  0. ]
 [3.  0.  0.  ... 0.  0.  0. ]
 [5.  0.  0.  ... 0.  0.  0. ]]


**Step 3: Finding the Best Rank k:**

The best rank $k$ is a matrix with prediction values; discuss this.

1. Use k = 50. I found the optimal rank 'k' for movie recomendation is another problem which can be the topic of my final project.

2. Computing SVD might be time consuming. If thats the case, discuss how to find an effcient algorithim.

3. From this matrix, construct the corresponding dataframe using: pd.DataFrame(prediction matrix, columns = original_dataframe.columns). This dataFrame will contain predicted ratings for movies by different users. Each row represents a user, and each column represents a movie, with the cells containing predicted ratings.


In [ ]:
# you code
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
reduced_data = pca.fit_transform(C)
predicted_ratings = np.dot(reduced_data, pca.components_)
predicted_df = pd.DataFrame(predicted_ratings, columns = ratings_2.columns)

**Step 4: Movie Recommendations:**
1. Pick a user retrieve its row in predictions and sort this in descending order (top-rated movies come first)

2. For the same user, retrieve it's original ratings and merge this information with the movies data frame to gather details about the movies the user has already rated. Store this combined information user_full.

3. Generate movie recommendations by merging the sorted predicted ratings with movie details and sorting the result by predicted ratings in descending order. The top-rated movies that the user hasn't seen yet are selected, and the specified number of recommendations is returned.





In [ ]:
# you code!

user_id = 10

user_predictions = predicted_df.iloc[user_id-1]
sorted_predictions = user_predictions.sort_values(ascending=False)
user_ratings = ratings_2.iloc[user_id-1]
user_full = pd.merge(user_ratings, movies, on='movieId')
recommendations = pd.merge(movies, sorted_predictions, left_on='movieId', right_index=True)
recommendations = recommendations.sort_values(by=sorted_predictions.name, ascending=False)

print(recommendations)

      movieId                                            title  \
7466    81845                        King's Speech, The (2010)   
7039    68954                                        Up (2009)   
4644     6942                             Love Actually (2003)   
3287     4447                            Legally Blonde (2001)   
5166     8368  Harry Potter and the Prisoner of Azkaban (2004)   
...       ...                                              ...   
507       589                Terminator 2: Judgment Day (1991)   
520       608                                     Fargo (1996)   
510       593                 Silence of the Lambs, The (1991)   
31         32        Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   
257       296                              Pulp Fiction (1994)   

                                  genres         9  
7466                               Drama  1.565031  
7039  Adventure|Animation|Children|Drama  1.436584  
4644                Comedy|Drama|Romance  1.3793

__Step 5__ From here, I wrote Python code that computes this for an arbitary user.

In [ ]:
# you code!

def recommend(user_id):

  user_predictions = predicted_df.iloc[user_id-1]
  sorted_predictions = user_predictions.sort_values(ascending=False)
  user_ratings = ratings_2.iloc[user_id-1]
  user_full = pd.merge(user_ratings, movies, on='movieId')
  recommendations = pd.merge(movies, sorted_predictions, left_on='movieId', right_index=True)
  recommendations = recommendations.sort_values(by=sorted_predictions.name, ascending=False)

  return recommendations

recommend(10)

,movieId,title,genres,9
7466,81845,"King's Speech, The (2010)",Drama,1.565031
7039,68954,Up (2009),Adventure|Animation|Children|Drama,1.436584
4644,6942,Love Actually (2003),Comedy|Drama|Romance,1.379356
3287,4447,Legally Blonde (2001),Comedy|Romance,1.251328
5166,8368,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy|IMAX,1.231959
...,...,...,...,...
507,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,-1.561157
520,608,Fargo (1996),Comedy|Crime|Drama|Thriller,-1.623929
510,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,-1.648378
31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,-1.662939


And that's the end of the lab!

__Note on Normalization:__
    Normalization is a statistical method used in various fields, including statistics, data analysis, and machine learning, to scale or transform data in a way that allows for meaningful comparisons and analysis. The specific techniques and purposes of normalization can vary, but the general goal is to standardize or rescale data to a common range or distribution.

In machine learning, it's common to normalize features to ensure that they have similar scales. This can improve the performance of many machine learning algorithms, such as gradient descent, which may converge faster and more reliably with normalized data.


Common methods of normalization include:

- **Min-Max Scaling:** This method scales the data to a specific range, often between 0 and 1. The formula for Min-Max scaling is `(x - min(x)) / (max(x) - min(x))`.

- **Z-Score Standardization:** This method standardizes the data to have a mean of 0 and a standard deviation of 1. It's also called standardization or mean normalization. The formula for Z-Score standardization is `(x - mean(x)) / std(x)`.

- **Log Transformation:** Taking the logarithm of data can be a form of normalization, especially when dealing with skewed or exponentially distributed data.

- **Box-Cox Transformation:** This is a family of power transformations that can stabilize variance and make data closer to a normal distribution.

- **Robust Scaling:** This method scales data using the median and interquartile range to handle outliers better.

The choice of normalization method depends on the specific context and data distribution. Normalization can be a crucial step in data preprocessing to ensure that data is suitable for analysis or machine learning models.

Refrences:

1. https://web.stanford.edu/class/cs168/l/l9.pdf

2. https://courses.cs.washington.edu/courses/cse521/16sp/521-lecture-9.pdf

3. https://beckernick.github.io/datascience/